# ETL Project: Google Play Store Apps

In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine, inspect
import numpy as np

# *Extract:*
----

### Kaggle Dataset: https://www.kaggle.com/lava18/google-play-store-apps?select=googleplaystore.csv

## CSV Files:
### - *googleplaystore.csv:*
    - Basic app details for over 8,000 apps on the Google Play Store
### - *googleplaystore_user_reviews.csv:* 
    - The first "most-relevant" 100 user reviews for apps on the Google Play Store through the letter 'H'


In [2]:
# Import two csv files
play_store = "Resources/googleplaystore.csv"
reviews = "Resources/googleplaystore_user_reviews.csv"

play_store_df = pd.read_csv(play_store)
reviews_df = pd.read_csv(reviews)

In [3]:
# View the Play Store DF
play_store_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,7-Jan-18,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,1-Aug-18,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,8-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up


In [4]:
# View the Reviews DF
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


# *Transform:*
---
## Data Cleaning & Transformation:
---
### - Drop any N/A rows

In [5]:
# Drop the NA values
play_store_df = play_store_df.dropna(how="any")
play_store_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,7-Jan-18,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,1-Aug-18,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,8-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up


In [6]:
# Drop the NA values
reviews_df = reviews_df.dropna(how="any")
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
5,10 Best Foods for You,Best way,Positive,1.00,0.300000


### - Drop unnecessary columns & keep the columns we want to use

In [7]:
# Drop unnecessary columns from play store DataFrame
del play_store_df["Category"]
del play_store_df["Size"]
del play_store_df["Type"]
del play_store_df["Last Updated"]
del play_store_df["Current Ver"]    
del play_store_df["Android Ver"]
play_store_df.head()

,App,Rating,Reviews,Installs,Price,Content Rating,Genres
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,"10,000+",0,Everyone,Art & Design
1,Coloring book moana,3.9,967,"500,000+",0,Everyone,Art & Design;Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,"5,000,000+",0,Everyone,Art & Design
3,Sketch - Draw & Paint,4.5,215644,"50,000,000+",0,Teen,Art & Design
4,Pixel Draw - Number Art Coloring Book,4.3,967,"100,000+",0,Everyone,Art & Design;Creativity


In [8]:
# Drop unnecessary columns from reviews DataFrame
del reviews_df["Translated_Review"]
reviews_df.head()

,App,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,Positive,1.00,0.533333
1,10 Best Foods for You,Positive,0.25,0.288462
3,10 Best Foods for You,Positive,0.40,0.875000
4,10 Best Foods for You,Positive,1.00,0.300000
5,10 Best Foods for You,Positive,1.00,0.300000


### - Drop Duplicates
### - Reset the Index
### - Create a Primary Key ID column
### - Set the ID column as the Index

In [9]:
# Drop the duplicate app names
play_store_df.drop_duplicates("App", inplace=True)
play_store_df = play_store_df.reset_index(drop=True).reset_index()
play_store_df = play_store_df.rename(columns={"index": "id"})
play_store_df.set_index("id", inplace=True)
play_store_df.head()

,App,Rating,Reviews,Installs,Price,Content Rating,Genres
id,,,,,,,
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,"10,000+",0,Everyone,Art & Design
1,Coloring book moana,3.9,967,"500,000+",0,Everyone,Art & Design;Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,"5,000,000+",0,Everyone,Art & Design
3,Sketch - Draw & Paint,4.5,215644,"50,000,000+",0,Teen,Art & Design
4,Pixel Draw - Number Art Coloring Book,4.3,967,"100,000+",0,Everyone,Art & Design;Creativity


In [10]:
# Drop the duplicate rows names
reviews_df.drop_duplicates(["Sentiment_Polarity","Sentiment_Subjectivity"], inplace=True)
reviews_df = reviews_df.reset_index(drop=True).reset_index()
reviews_df = reviews_df.rename(columns={"index": "id"})
reviews_df.set_index("id", inplace=True)
reviews_df.head()

,App,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
id,,,,
0,10 Best Foods for You,Positive,1.00,0.533333
1,10 Best Foods for You,Positive,0.25,0.288462
2,10 Best Foods for You,Positive,0.40,0.875000
3,10 Best Foods for You,Positive,1.00,0.300000
4,10 Best Foods for You,Positive,0.60,0.900000


### - Rename columns to match the SQL database tables

In [11]:
# Rename the columns for reviews df
reviews_df = reviews_df.rename(columns={"App": "app", "Sentiment": "sentiment", "Sentiment_Polarity": "sentiment_polarity", 
                                        "Sentiment_Subjectivity": "sentiment_subjectivity"})
reviews_df.head()

,app,sentiment,sentiment_polarity,sentiment_subjectivity
id,,,,
0,10 Best Foods for You,Positive,1.00,0.533333
1,10 Best Foods for You,Positive,0.25,0.288462
2,10 Best Foods for You,Positive,0.40,0.875000
3,10 Best Foods for You,Positive,1.00,0.300000
4,10 Best Foods for You,Positive,0.60,0.900000


In [12]:
# Rename the columns for play store df
play_store_df = play_store_df.rename(columns={"App": "app", "Rating": "rating", "Reviews": "reviews", 
                                              "Installs": "installs", "Price": "price", 
                                              "Content Rating": "content_rating", "Genres": "genres"})
play_store_df.head()

,app,rating,reviews,installs,price,content_rating,genres
id,,,,,,,
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,"10,000+",0,Everyone,Art & Design
1,Coloring book moana,3.9,967,"500,000+",0,Everyone,Art & Design;Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,"5,000,000+",0,Everyone,Art & Design
3,Sketch - Draw & Paint,4.5,215644,"50,000,000+",0,Teen,Art & Design
4,Pixel Draw - Number Art Coloring Book,4.3,967,"100,000+",0,Everyone,Art & Design;Creativity


### - Re-format the Installs column to remove the "+" and the ","

In [13]:
# Remove the plus sign & commas on Installs, change data type to int
play_store_df["installs"] = play_store_df["installs"].str.replace('+', '')
play_store_df["installs"] = play_store_df["installs"].str.replace(',', '').astype(int)

# Remove the $ and change the data type to float
play_store_df["price"] = play_store_df["price"].str.replace('$', '').astype(float)

play_store_df.head()

,app,rating,reviews,installs,price,content_rating,genres
id,,,,,,,
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,10000,0.0,Everyone,Art & Design
1,Coloring book moana,3.9,967,500000,0.0,Everyone,Art & Design;Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,5000000,0.0,Everyone,Art & Design
3,Sketch - Draw & Paint,4.5,215644,50000000,0.0,Teen,Art & Design
4,Pixel Draw - Number Art Coloring Book,4.3,967,100000,0.0,Everyone,Art & Design;Creativity


### - Change the data types to reflect the data types used in the SQL database

In [14]:
# Change the data type of reviews to int
play_store_df["reviews"] = play_store_df["reviews"].astype(int)

### - Export DataFrames to CSV files for future use

In [15]:
# Export csvs
play_store_df.to_csv("Output/play_store.csv")
reviews_df.to_csv("Output/reviews.csv")

# *Load:*
---
## Relational Database: SQL
### Why Relational?
It works well with the datasets we chose: The "app" column in the "Reviews" dataset relates to the "app" column in the "Play Store (Apps)" dataset in a "one-to-many" relationship

## Connect to SQL Database:
---
### 1. Create a new database: "google_play_store_db" in pgAdmin 4
### 2. Use SQL code from "Resources/google_play.sql" to create tables in the new database 
### 3. Connect to the database using SQL Alchemy (code below)

In [16]:
# Connect to the sql database
connection_string = "postgres:postgres@localhost:5432/google_play_store_db"
engine = create_engine(f'postgresql://{connection_string}')

## Table Schemas:
---
### apps:
- id (PK - INT)
- app (VARCHAR)
- rating (DECIMAL)
- reviews (INT)
- installs (INT)
- price (DECIMAL)
- content_rating (VARCHAR)
- genres (VARCHAR)
---
### reviews:
- id (PK - INT)
- app (VARCHAR)
- sentiment (VARCHAR)
- sentiment_polarity (DECIMAL)
- sentiment_subjectivity (DECIMAL)
- app_id (FK - apps (id) ) (INT)

In [17]:
# Get the table names
Inspector = inspect(engine)
Inspector.get_table_names()

['apps', 'reviews']

In [18]:
# RESET: Drop both tables
engine.execute("DROP TABLE reviews")
engine.execute("DROP TABLE apps")

### - Load the cleaned DataFrames to the SQL Database

In [19]:
# Append the dataframes to sql database
play_store_df.to_sql(name='apps', con=engine, if_exists='append', index=True)
reviews_df.to_sql(name='reviews', con=engine, if_exists='append', index=True)

### - Check the data was loaded correctly by querying the database

In [20]:
# Query the database (apps)
engine.execute(
    """SELECT *
    FROM apps
    LIMIT 10""").fetchall()

[(0, 'Photo Editor & Candy Camera & Grid & ScrapBook', 4.1, 159, 10000, 0.0, 'Everyone', 'Art & Design'),
 (1, 'Coloring book moana', 3.9, 967, 500000, 0.0, 'Everyone', 'Art & Design;Pretend Play'),
 (2, 'U Launcher Lite – FREE Live Cool Themes, Hide Apps', 4.7, 87510, 5000000, 0.0, 'Everyone', 'Art & Design'),
 (3, 'Sketch - Draw & Paint', 4.5, 215644, 50000000, 0.0, 'Teen', 'Art & Design'),
 (4, 'Pixel Draw - Number Art Coloring Book', 4.3, 967, 100000, 0.0, 'Everyone', 'Art & Design;Creativity'),
 (5, 'Paper flowers instructions', 4.4, 167, 50000, 0.0, 'Everyone', 'Art & Design'),
 (6, 'Smoke Effect Photo Maker - Smoke Editor', 3.8, 178, 50000, 0.0, 'Everyone', 'Art & Design'),
 (7, 'Infinite Painter', 4.1, 36815, 1000000, 0.0, 'Everyone', 'Art & Design'),
 (8, 'Garden Coloring Book', 4.4, 13791, 1000000, 0.0, 'Everyone', 'Art & Design'),
 (9, 'Kids Paint Free - Drawing Fun', 4.7, 121, 10000, 0.0, 'Everyone', 'Art & Design;Creativity')]

In [21]:
# Query the database (reviews)
engine.execute(
    """SELECT *
    FROM reviews
    LIMIT 10""").fetchall()

[(0, '10 Best Foods for You', 'Positive', 1.0, 0.533333333),
 (1, '10 Best Foods for You', 'Positive', 0.25, 0.288461538),
 (2, '10 Best Foods for You', 'Positive', 0.4, 0.875),
 (3, '10 Best Foods for You', 'Positive', 1.0, 0.3),
 (4, '10 Best Foods for You', 'Positive', 0.6, 0.9),
 (5, '10 Best Foods for You', 'Neutral', 0.0, 0.0),
 (6, '10 Best Foods for You', 'Positive', 0.7, 0.6),
 (7, '10 Best Foods for You', 'Positive', 0.2, 0.1),
 (8, '10 Best Foods for You', 'Positive', 0.75, 0.875),
 (9, '10 Best Foods for You', 'Positive', 0.9921875, 0.8666666670000001)]

## Create a Foreign ID column in Reviews Table:

### - Join the tables on "app" to find app ids & other column info

In [22]:
# Select app name and id to find ids to make foreign ids in reviews table
foreign_ids = engine.execute(
    """SELECT r.app, r.sentiment, r.sentiment_polarity, r.sentiment_subjectivity, a.id
    from apps as a
    inner join reviews as r on
    a.app = r.app;""").fetchall()

### - Convert the query results to a Foreign ID DataFrame, rename columns, and reset the index

In [23]:
# Create a foreign id for reviews_df
foreign_id_df = pd.DataFrame(foreign_ids)
foreign_id_df = foreign_id_df.rename(columns={0: "app", 1: "sentiment", 2: "sentiment_polarity", 
                                              3: "sentiment_subjectivity", 4: "app_id"})
foreign_id_df = foreign_id_df.reset_index(drop=True).reset_index()
foreign_id_df = foreign_id_df.rename(columns={"index": "id"})
foreign_id_df.set_index("id", inplace=True)

foreign_id_df.head()

,app,sentiment,sentiment_polarity,sentiment_subjectivity,app_id
id,,,,,
0,10 Best Foods for You,Positive,0.4,0.875000,1078
1,10 Best Foods for You,Positive,1.0,0.300000,1078
2,10 Best Foods for You,Positive,1.0,0.533333,1078
3,10 Best Foods for You,Positive,0.6,0.900000,1078
4,10 Best Foods for You,Neutral,0.0,0.000000,1078


### - Reset the Reviews table in the Database and add the new data (with the new FK column)

In [24]:
# Drop the reviews table: reset
engine.execute("DROP TABLE reviews")
foreign_id_df.to_sql(name='reviews', con=engine, if_exists='append', index=True)

# Query the database again
engine.execute(
    """SELECT *
    FROM reviews
    LIMIT 10""").fetchall()

[(0, '10 Best Foods for You', 'Positive', 0.4, 0.875, 1078),
 (1, '10 Best Foods for You', 'Positive', 1.0, 0.3, 1078),
 (2, '10 Best Foods for You', 'Positive', 1.0, 0.533333333, 1078),
 (3, '10 Best Foods for You', 'Positive', 0.6, 0.9, 1078),
 (4, '10 Best Foods for You', 'Neutral', 0.0, 0.0, 1078),
 (5, '10 Best Foods for You', 'Positive', 0.7, 0.6, 1078),
 (6, '10 Best Foods for You', 'Positive', 0.2, 0.1, 1078),
 (7, '10 Best Foods for You', 'Positive', 0.75, 0.875, 1078),
 (8, '10 Best Foods for You', 'Positive', 0.9921875, 0.8666666670000001, 1078),
 (9, '10 Best Foods for You', 'Positive', 0.55, 0.5111111110000001, 1078)]

### - Join the tables to check the Foreign Keys 

In [25]:
# Join the two tables to get query results
# Greatest Number of Reviews
engine.execute(
    """SELECT a.id, a.app, count(r.app_id) as num_reviews
    FROM apps as a
    INNER JOIN reviews as r on
    a.id = r.app_id
    GROUP BY a.id, a.app
    ORDER BY num_reviews DESC
    LIMIT 10""").fetchall()

[(1300, 'Angry Birds Classic', 94),
 (1280, 'Clash Royale', 91),
 (1295, '8 Ball Pool', 90),
 (1870, 'Facebook', 89),
 (1446, "Alto's Adventure", 89),
 (1310, 'Gardenscapes', 87),
 (1081, 'Calorie Counter - Macros', 83),
 (1003, '8fit Workouts & Meal Planner', 83),
 (1454, 'DINO HUNTER: DEADLY SHORES', 81),
 (2655, 'ColorNote Notepad Notes', 80)]

# Final Database Tables:

In [26]:
# Final Reviews Table with Foreign Key that connects to the Apps Table
engine.execute(
    """SELECT *
    FROM reviews
    LIMIT 10""").fetchall()

[(0, '10 Best Foods for You', 'Positive', 0.4, 0.875, 1078),
 (1, '10 Best Foods for You', 'Positive', 1.0, 0.3, 1078),
 (2, '10 Best Foods for You', 'Positive', 1.0, 0.533333333, 1078),
 (3, '10 Best Foods for You', 'Positive', 0.6, 0.9, 1078),
 (4, '10 Best Foods for You', 'Neutral', 0.0, 0.0, 1078),
 (5, '10 Best Foods for You', 'Positive', 0.7, 0.6, 1078),
 (6, '10 Best Foods for You', 'Positive', 0.2, 0.1, 1078),
 (7, '10 Best Foods for You', 'Positive', 0.75, 0.875, 1078),
 (8, '10 Best Foods for You', 'Positive', 0.9921875, 0.8666666670000001, 1078),
 (9, '10 Best Foods for You', 'Positive', 0.55, 0.5111111110000001, 1078)]

In [27]:
# Final Apps Table
engine.execute(
    """SELECT *
    FROM apps
    LIMIT 10""").fetchall()

[(0, 'Photo Editor & Candy Camera & Grid & ScrapBook', 4.1, 159, 10000, 0.0, 'Everyone', 'Art & Design'),
 (1, 'Coloring book moana', 3.9, 967, 500000, 0.0, 'Everyone', 'Art & Design;Pretend Play'),
 (2, 'U Launcher Lite – FREE Live Cool Themes, Hide Apps', 4.7, 87510, 5000000, 0.0, 'Everyone', 'Art & Design'),
 (3, 'Sketch - Draw & Paint', 4.5, 215644, 50000000, 0.0, 'Teen', 'Art & Design'),
 (4, 'Pixel Draw - Number Art Coloring Book', 4.3, 967, 100000, 0.0, 'Everyone', 'Art & Design;Creativity'),
 (5, 'Paper flowers instructions', 4.4, 167, 50000, 0.0, 'Everyone', 'Art & Design'),
 (6, 'Smoke Effect Photo Maker - Smoke Editor', 3.8, 178, 50000, 0.0, 'Everyone', 'Art & Design'),
 (7, 'Infinite Painter', 4.1, 36815, 1000000, 0.0, 'Everyone', 'Art & Design'),
 (8, 'Garden Coloring Book', 4.4, 13791, 1000000, 0.0, 'Everyone', 'Art & Design'),
 (9, 'Kids Paint Free - Drawing Fun', 4.7, 121, 10000, 0.0, 'Everyone', 'Art & Design;Creativity')]